# **Financial Applicactions with ML & AI**

<img style="float: right;" src="https://github.com/torreblanca99/course_financial_applications/blob/develop/docs/img/logo_bourbaki.png?raw=1" width="100"/>

## **Module I:** Default Analysis
#### Topic:  Clasification with Linear Discriminant Analysis

##### Name: Julio César Avila Torreblanca

- **Objective**: apply linear discriminant analysis to a default classification problem.

- **Contents**:
    - Notes:
        - LDA Algorithm
    - Code:
        1. Libraries and parameters
        2. Read Data
        3. EDA
        4. Modeling
        5. Evaluation
        6. Conclusions

----


# NOTES: Linear Discriminant Analysis for Default Classification

### Introduction to Linear Discriminant Analysis

Linear Discriminant Analysis (LDA) is a supervised machine learning algorithm that is primarily used for classification purposes. It seeks to find a linear combination of features that characterizes or separates two or more classes (in this case, defaulters and non-defaulters).

### Purpose of LDA in Default Classification

In the context of financial applications, LDA can be employed to classify whether a client will default or not on a loan based on historical data. The algorithm helps in reducing dimensionality while preserving as much class discriminatory information as possible.

### Mathematical Formulation

LDA works by modeling the difference between the classes as linear equations. It aims to maximize the ratio of between-class variance to the within-class variance, thereby ensuring maximum separability. The mathematical formulation involves:

1. Compute the mean vectors for each class.
2. Compute the within-class scatter matrix ($S_W$):

$$S_W = \sum_{i=1}^{c} \sum_{x \in D_i} (x - \mu_i)(x - \mu_i)^T$$

3. Compute the between-class scatter matrix ($S_B$):
$$S_B = \sum_{i=1}^{c} N_i (\mu_i - \mu)(\mu_i - \mu)^T$$

4. Solve the generalized eigenvalue problem for the matrix ($S_W^{-1}S_B$).

The solution of this problem gives us the linear discriminant coefficients.

### Implementation Steps

1. Standardize the dataset.
2. Compute the LDA components.
3. Project the data onto the LDA components.
4. Use the projections for classification.

### Advantages of LDA

- **Dimensionality Reduction**: LDA can reduce the number of features needed while retaining the class discriminatory information.
- **Computationally Efficient**: Compared to other classification methods like SVM and neural networks, LDA is relatively fast.
- **Interpretability**: The linear combinations of features are easy to interpret.
- **Captures Class Covariance**: Takes into account the covariance between the different classes.

### Disadvantages of LDA

- **Assumption of Normality**: LDA assumes that data is normally distributed, which may not hold true in many real-world applications.
- **Linearity Assumption**: It assumes linear decision boundaries, which may not capture complex relationships well.
- **Sensitive to Outliers**: Being based on the mean, LDA is sensitive to outliers which can mislead the results.
- **Not Suitable for Non-Gaussian Distributions**: If the Gaussian assumption is seriously violated, LDA will perform poorly.

---


# Coding

# 1. Libraries and Parameters

In [11]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix

import plotly.express as px

# 2. Data reading

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel("/content/drive/MyDrive/Cruso-ApsFinancieras/semana1/lending_clubFull_Data_Set.xlsx", index_col=0)
df

,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,263591,545710,20000.0,60 months,17.93,342.94,E,E5,Wylie ISD,1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
1,1613916,69664096,30000.0,36 months,11.99,996.29,C,C1,Sergeant,10+ years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
2,818934,8965180,21500.0,36 months,11.99,714.01,B,B3,Designer,1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
3,1606612,70572960,10000.0,36 months,13.67,340.18,C,C3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
4,1639932,68589517,5000.0,36 months,8.49,157.82,B,B1,Sr. Manufacturing Engineer,10+ years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,901888,4974773,10500.0,36 months,10.16,339.60,B,B1,Schneider Electric,5 years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
24996,945413,1279809,12000.0,36 months,14.33,412.06,C,C1,Clark County School District,10+ years,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
24997,366944,63496281,25000.0,36 months,12.69,838.63,C,C2,sales consultant,< 1 year,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN
24998,1043529,98124387,12200.0,60 months,13.49,280.66,C,C2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


# 3. EDA

## 3.1 Miss values

In [5]:
df.isna().sum().sort_values()

,0
id,0
member_id,0
initial_list_status,1
last_fico_range_low,1
revol_bal,1
...,...
hardship_status,24904
hardship_reason,24904
hardship_type,24904
deferral_term,24904


In [6]:
na_values = (df.isna().sum().sort_values() / len(df)).reset_index(name = 'n')
na_values

,index,n
0,id,0.00000
1,member_id,0.00000
2,initial_list_status,0.00004
3,last_fico_range_low,0.00004
4,revol_bal,0.00004
...,...,...
129,hardship_status,0.99616
130,hardship_reason,0.99616
131,hardship_type,0.99616
132,deferral_term,0.99616


In [7]:
aux = na_values[na_values['n']>0.1]
aux

,index,n
76,mths_since_recent_inq,0.13516
77,next_pymnt_d,0.49164
78,mths_since_last_delinq,0.49172
79,open_acc_6m,0.52096
80,open_act_il,0.52096
81,inq_last_12m,0.52096
82,open_il_12m,0.52096
83,open_il_24m,0.52096
84,total_bal_il,0.52096
85,open_rv_24m,0.52096


In [8]:
columns_to_drop = list(aux['index'])
columns_to_drop

['mths_since_recent_inq',
 'next_pymnt_d',
 'mths_since_last_delinq',
 'open_acc_6m',
 'open_act_il',
 'inq_last_12m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'open_rv_24m',
 'max_bal_bc',
 'inq_fi',
 'total_cu_tl',
 'open_rv_12m',
 'all_util',
 'mths_since_rcnt_il',
 'il_util',
 'mths_since_recent_revol_delinq',
 'mths_since_last_major_derog',
 'mths_since_recent_bc_dlq',
 'mths_since_last_record',
 'desc',
 'dti_joint',
 'verification_status_joint',
 'annual_inc_joint',
 'sec_app_open_act_il',
 'sec_app_collections_12_mths_ex_med',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_num_rev_accts',
 'sec_app_open_acc',
 'sec_app_fico_range_high',
 'sec_app_inq_last_6mths',
 'sec_app_earliest_cr_line',
 'sec_app_fico_range_low',
 'revol_bal_joint',
 'sec_app_mort_acc',
 'sec_app_revol_util',
 'settlement_amount',
 'settlement_status',
 'settlement_date',
 'debt_settlement_flag_date',
 'settlement_percentage',
 'settlement_term',
 'sec_app_mths_since_last_major_derog',
 'hardship_

In [9]:
len(columns_to_drop)

58

In [12]:
px.bar(na_values, "index", "n")

In [13]:
df_clean = df.drop(columns=columns_to_drop)
df_clean

,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag
0,263591,545710,20000.0,60 months,17.93,342.94,E,E5,Wylie ISD,1 year,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N
1,1613916,69664096,30000.0,36 months,11.99,996.29,C,C1,Sergeant,10+ years,...,1.0,90.0,25.0,1.0,0.0,116075.0,69594.0,27500.0,83175.0,N
2,818934,8965180,21500.0,36 months,11.99,714.01,B,B3,Designer,1 year,...,1.0,100.0,50.0,0.0,0.0,116920.0,74939.0,19700.0,78020.0,N
3,1606612,70572960,10000.0,36 months,13.67,340.18,C,C3,NaN,NaN,...,3.0,80.0,50.0,0.0,0.0,39547.0,16824.0,11200.0,25547.0,N
4,1639932,68589517,5000.0,36 months,8.49,157.82,B,B1,Sr. Manufacturing Engineer,10+ years,...,0.0,60.0,100.0,0.0,0.0,104310.0,7582.0,3300.0,13015.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,901888,4974773,10500.0,36 months,10.16,339.60,B,B1,Schneider Electric,5 years,...,1.0,96.0,25.0,0.0,0.0,263503.0,40380.0,13000.0,33279.0,N
24996,945413,1279809,12000.0,36 months,14.33,412.06,C,C1,Clark County School District,10+ years,...,3.0,100.0,40.0,0.0,0.0,64327.0,64497.0,5400.0,51027.0,N
24997,366944,63496281,25000.0,36 months,12.69,838.63,C,C2,sales consultant,< 1 year,...,1.0,90.2,33.3,0.0,0.0,183700.0,22014.0,3600.0,0.0,N
24998,1043529,98124387,12200.0,60 months,13.49,280.66,C,C2,NaN,NaN,...,2.0,100.0,33.3,0.0,0.0,22500.0,13615.0,20000.0,1000.0,N


## 3.2 Analysis by data type

### Numerical

# 4. Modeling

In [ ]:
b

# 5. Model Evaluation

NameError: name 'v' is not defined

# 4. Conclusions